Question 1: What is the role of filters and feature maps in Convolutional Neural Network (CNN)?

In Convolutional Neural Networks (CNNs), filters (also called kernels) and feature maps are core components that enable the network to learn and extract spatial features from input data like images.

Filters: These are small, learnable matrices (e.g., 3x3 or 5x5) that slide over the input image or previous feature map, performing element-wise multiplications and summations to detect patterns such as edges, textures, or shapes. Each filter learns to recognize specific features during training via backpropagation. For example, an edge-detecting filter might have weights that emphasize differences between adjacent pixels. Filters promote parameter sharing, reducing the model's complexity compared to fully connected layers—evidence from architectures like LeNet-5 shows this leads to efficient learning of hierarchical features.

Feature Maps: These are the outputs of applying a filter to the input. Each filter produces one feature map, which is a 2D array highlighting where the filter's pattern is present in the input. Multiple filters generate multiple feature maps (e.g., 32 filters on a 28x28 input might yield 32 feature maps). They capture activations at different spatial locations, allowing the network to build representations from low-level (e.g., edges) to high-level (e.g., objects) features. In practice, feature maps from deeper layers become more abstract, as seen in AlexNet's layers, where early maps detect edges and later ones recognize complex shapes, improving classification accuracy on datasets like ImageNet.

Together, they make CNNs translation-invariant and computationally efficient, with empirical studies (e.g., in Zeiler & Fergus, 2014) visualizing feature maps to confirm their role in feature extraction.

Question 2: Explain the concepts of padding and stride in CNNs. How do they affect the output dimensions of feature maps?

Padding and stride are hyperparameters in convolutional layers that control how filters are applied to inputs, directly influencing the spatial dimensions of output feature maps.

Padding: This adds extra pixels (usually zeros) around the input image or feature map to preserve spatial information and prevent shrinking. For example, "same" padding ensures the output size matches the input, while "valid" padding (no padding) allows shrinking. Padding helps maintain border features, as filters can center on edge pixels.

Stride: This defines how many pixels the filter moves (slides) at each step (e.g., stride 1 moves one pixel, stride 2 skips every other). Larger strides reduce the output size faster, introducing subsampling.

Effect on Output Dimensions: For an input of height H, width W, filter size F, padding P, and stride S, the output dimensions are:

Output Height:

Output Width:

For instance, a 28x28 input with a 3x3 filter, stride 1, and padding 1 yields 28x28 output (preserving size). Without padding (P=0), it becomes 26x26. Larger strides (e.g., S=2) halve dimensions, reducing parameters but potentially losing detail. This is crucial for controlling receptive fields and computational cost, as seen in VGG nets using stride 1 for fine-grained features and pooling for downsampling.

Question 3: Define receptive field in the context of CNNs. Why is it important for deep architectures?

The receptive field of a neuron in a CNN is the region of the input image that influences its activation. It grows with depth: for a neuron in layer L, it's determined by the filter sizes and strides of preceding layers. For example, in a simple CNN with 3x3 filters and stride 1, a neuron in layer 3 has a 7x7 receptive field.

It's important for deep architectures because larger receptive fields allow neurons to capture global context (e.g., entire objects), enabling better feature integration. Without sufficient size, networks struggle with complex patterns, leading to poor performance—as evidenced by ResNet's design, where deeper layers have expansive fields for tasks like ImageNet classification. However, overly large fields can increase parameters; techniques like dilated convolutions expand fields without more layers.

Question 4: Discuss how filter size and stride influence the number of parameters in a CNN.

Filter size and stride affect parameter count by determining connectivity and output size, impacting model complexity and efficiency.

Filter Size (F): Larger filters (e.g., 7x7 vs. 3x3) increase parameters per layer, as each filter has F×F weights (plus bias). For C input channels and K filters, parameters = K × (F×F × C + 1). Larger F captures broader patterns but risks overfitting; smaller F (e.g., 3x3 in VGG) allows deeper stacks with fewer parameters overall, as shown by VGG-16's ~138M params vs. larger-filter alternatives.

Stride (S): Larger strides reduce output feature map size, indirectly lowering parameters in subsequent layers by decreasing input dimensions. For example, stride 2 halves the map, cutting params in the next conv layer. This trades resolution for efficiency, as in AlexNet's pooling-equivalent strides, reducing compute while maintaining performance on ImageNet.

Overall, smaller filters and moderate strides minimize parameters (e.g., MobileNet uses 1x1 and 3x3 for efficiency), but optimal choices balance expressiveness and overfitting, per empirical tuning in papers like He et al. (2016).

Question 5: Compare and contrast different CNN-based architectures like LeNet, AlexNet, and VGG in terms of depth, filter sizes, and performance.

LeNet, AlexNet, and VGG are foundational CNNs, evolving in depth, filter sizes, and performance for image classification.

Depth: LeNet-5 has 5 layers (2 conv, 3 FC). AlexNet has 8 layers (5 conv, 3 FC), deeper due to GPUs. VGG-16/19 has 16-19 layers (13-16 conv, 3 FC), enabling hierarchical learning but risking vanishing gradients—ResNet addressed this later.

Filter Sizes: LeNet uses 5x5 filters. AlexNet mixes 11x11 (first layer) and 5x5/3x3, with large initial filters for broad features. VGG standardizes 3x3 filters throughout, stacking them for effective receptive fields (e.g., two 3x3 = 5x5 coverage), reducing params.

Performance: LeNet achieved ~99% on MNIST (1998). AlexNet dropped ImageNet top-5 error to 15.3% (2012), using ReLU and dropout. VGG improved to 6.8% (2014) with depth, but slower training. VGG excels in transfer learning; AlexNet pioneered GPU training; LeNet laid basics. All outperform shallow nets, but VGG's uniformity aids fine-tuning, per Simonyan et al. (2014).

Question 6: Using Keras, build and train a simple CNN model on the MNIST dataset from scratch. Include code for module creation, compilation, training, and evaluation.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data
# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape to add channel dimension (grayscale images)
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

# Convert labels to categorical (one-hot encoding)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 47s 54ms/step - accuracy: 0.8706 - loss: 0.4285 - val_accuracy: 0.9812 - val_loss: 0.0663
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 45s 54ms/step - accuracy: 0.9820 - loss: 0.0593 - val_accuracy: 0.9873 - val_loss: 0.0451
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 53s 62ms/step - accuracy: 0.9888 - loss: 0.0373 - val_accuracy: 0.9845 - val_loss: 0.0537
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 74s 53ms/step - accuracy: 0.9907 - loss: 0.0294 - val_accuracy: 0.9873 - val_loss: 0.0445
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 81s 53ms/step - accuracy: 0.9938 - loss: 0.0204 - val_accuracy: 0.9890 - val_loss: 0.0395
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9865 - loss: 0.0367
Test accuracy: 0.9894


 Question 7: Load and preprocess the CIFAR-10 dataset using Keras, and create a
CNN model to classify RGB images. Show your preprocessing and architecture.

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Preprocess the data
# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical (one-hot encoding)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Build the CNN model for RGB images (3 channels)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 71s 98ms/step - accuracy: 0.3202 - loss: 1.8317 - val_accuracy: 0.5018 - val_loss: 1.3646
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 65s 92ms/step - accuracy: 0.5439 - loss: 1.2833 - val_accuracy: 0.5990 - val_loss: 1.1352
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 81s 91ms/step - accuracy: 0.6133 - loss: 1.1052 - val_accuracy: 0.6322 - val_loss: 1.0498
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 65s 92ms/step - accuracy: 0.6569 - loss: 0.9768 - val_accuracy: 0.6428 - val_loss: 1.0458
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.6889 - loss: 0.8964 - val_accuracy: 0.6900 - val_loss: 0.9098
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 84s 93ms/step - accuracy: 0.7114 - loss: 0.8277 - val_accuracy: 0.6888 - val_loss: 0.9268
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 64s 91ms/step - accuracy: 0.7405 - loss: 0.7474 - val_accuracy: 0.6952 - val_loss: 0.9128
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━

Question 8: Using PyTorch, write a script to define and train a CNN on the MNIST
dataset. Include model definition, data loaders, training loop, and accuracy evaluation.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Data preprocessing and loaders
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model, loss, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

100%|██████████| 9.91M/9.91M [00:03<00:00, 2.87MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 158kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.52MB/s]


Epoch 1/5, Loss: 0.2529
Epoch 2/5, Loss: 0.0895
Epoch 3/5, Loss: 0.0666
Epoch 4/5, Loss: 0.0540
Epoch 5/5, Loss: 0.0456
Accuracy on test set: 99.15%


Question 9: Given a custom image dataset stored in a local directory, write code using
Keras ImageDataGenerator to preprocess and train a CNN model.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from PIL import Image

# Define paths (replace with your actual paths)
train_dir = './data/train'  # Example: relative path to your training dataset directory
validation_dir = './data/validation'  # Example: relative path to your validation dataset directory (or None if using split)

# --- Start of fix: Create dummy directories and files for demonstration ---
# This section creates a minimal directory structure with dummy images
# so that flow_from_directory can run without FileNotFoundError.
# You should remove or comment out this section when using your actual dataset.

def create_dummy_image(filepath, size=(150, 150)):
    image = Image.fromarray(np.random.randint(0, 255, (size[0], size[1], 3), dtype=np.uint8))
    image.save(filepath)

# Create base data directory if it doesn't exist
os.makedirs('./data', exist_ok=True)

# Create train and validation directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Define dummy classes
dummy_classes = ['class_a', 'class_b']

# Create class subdirectories and dummy images in train_dir
for class_name in dummy_classes:
    class_path = os.path.join(train_dir, class_name)
    os.makedirs(class_path, exist_ok=True)
    for i in range(5):  # Create 5 dummy images per class
        create_dummy_image(os.path.join(class_path, f'train_image_{class_name}_{i}.png'))

# Create class subdirectories and dummy images in validation_dir
for class_name in dummy_classes:
    class_path = os.path.join(validation_dir, class_name)
    os.makedirs(class_path, exist_ok=True)
    for i in range(2):  # Create 2 dummy images per class
        create_dummy_image(os.path.join(class_path, f'val_image_{class_name}_{i}.png'))

print("Dummy directories and images created for demonstration.")

# --- End of fix ---


# ImageDataGenerator for preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotations
    width_shift_range=0.2,  # Random horizontal shifts
    height_shift_range=0.2,  # Random vertical shifts
    shear_range=0.2,  # Random shearing
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flips
    fill_mode='nearest',  # Fill mode for new pixels
    validation_split=0.2  # Optional: use 20% of training data for validation if no separate dir
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation

# Data loaders
if validation_dir and os.path.exists(validation_dir):
    # Use separate validation directory
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),  # Resize images to 150x150
        batch_size=32,
        class_mode='categorical'  # For multi-class; use 'binary' for binary
    )
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical'
    )
else:
    # Use validation split from training data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        subset='training'  # Use for training
    )
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        subset='validation'  # Use for validation
    )

# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_accuracy:.4f}")

Dummy directories and images created for demonstration.
Found 10 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 8.0946
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 848ms/step - accuracy: 0.5000 - loss: 6.5292 - val_accuracy: 0.5000 - val_loss: 1.1573
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 811ms/step - accuracy: 0.5000 - loss: 0.9758 - val_accuracy: 0.5000 - val_loss: 0.9181
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.8222 - val_accuracy: 0.5000 - val_loss: 0.6901
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.6912 - val_accuracy: 0.5000 - val_loss: 0.7252
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.7103 - val_accuracy: 0.5000 - val_loss: 0.6942
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.6922 - val_accuracy: 0.5000 - val_loss: 0.6944
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.6949 - val_accuracy: 0.5000 - val_loss: 0.7001
Epoch 9/10
1/1 ━━━━━━

Question 10: You are working on a web application for a medical imaging startup. Your
task is to build and deploy a CNN model that classifies chest X-ray images into “Normal”
and “Pneumonia” categories. Describe your end-to-end approach–from data preparation
and model training to deploying the model as a web app using Streamlit.

1. Data Preparation
Dataset Acquisition: Download the Chest X-Ray Images dataset (e.g., from Kaggle). It includes ~5,800 training images and ~1,600 test images, split into "NORMAL" and "PNEUMONIA" folders. Use libraries like os and shutil to organize data into directories.
Preprocessing:
Resize images to a consistent size (e.g., 224x224 pixels) using tf.image.resize to standardize input for the CNN.
Normalize pixel values to [0, 1] by dividing by 255.
Apply data augmentation (e.g., random rotations, flips) using ImageDataGenerator to prevent overfitting and improve generalization, especially since pneumonia images may be imbalanced.
Split data: 80% training, 10% validation, 10% test. Use train_test_split or Keras' flow_from_directory for efficient loading.
Handling Imbalances: If pneumonia cases dominate, use class weights during training (e.g., via class_weight in model.fit).
2. Model Building and Training
Architecture: Build a simple CNN using Keras for binary classification:
Input: 224x224x3 RGB images.
Layers: Conv2D (32 filters, 3x3 kernel, ReLU), MaxPooling2D (2x2), Conv2D (64 filters), MaxPooling2D, Conv2D (128 filters), MaxPooling2D, Flatten, Dense (128 units, ReLU), Dense (1 unit, Sigmoid) for binary output.
This is lightweight (~1M parameters) and effective for image classification tasks like this.
Compilation: Use Adam optimizer, binary cross-entropy loss, and accuracy/metrics like AUC-ROC for evaluation.
Training: Train for 10-20 epochs with batch size 32, using early stopping (monitor validation loss) to avoid overfitting. Expect ~85-95% accuracy on test data, based on similar models in literature (e.g., studies on this dataset achieve high performance with CNNs).
Evaluation: Assess with confusion matrix, precision, recall, and F1-score. Visualize training curves using Matplotlib to ensure convergence.

3. Model Deployment as a Web App Using Streamlit
Save the Model: After training, save the model as an HDF5 file (model.save('chest_xray_model.h5')) for easy loading.
Build the Streamlit App:
Install Streamlit (pip install streamlit).
Create app.py: Load the model, add a file uploader for X-ray images, preprocess uploaded images (resize/normalize), make predictions, and display results (e.g., "Normal" or "Pneumonia" with confidence score).
Include UI elements like titles, descriptions, and warnings (e.g., "This is not a substitute for medical advice").
Deployment:
Run locally with streamlit run app.py.
For production, deploy on platforms like Streamlit Cloud, Heroku, or AWS. Upload the model file and ensure dependencies (e.g., TensorFlow) are specified in requirements.txt.
Add security: Limit file types to images, validate inputs, and consider authentication for sensitive data.
Testing and Iteration: Test the app with sample images, gather feedback, and retrain the model periodically with new data for robustness.